# Precomputing values for use in fits of Stan models

Because of the way Stan works, it is necessary to compute some values in advance which can then be passed into the fit an interpolated over. The precomputed values will be different for different sets of source distances, and therefore different catalogues. 

Here we show how to compute the values for the SBG catalogue, but it is exactly the same for all cases, just changing the input label.

For ease, all the precomputed table files used are provided for use in this repository.

In [1]:
import os
import h5py
import numpy as np

from fancy import Data, Model, Analysis

# Define file containing catalogue information
source_file = '../data/sourcedata.h5'

# Path to Stan files
stan_path = '../stan/'

# source_types = ["SBG_23", "2FHL_250Mpc", "swift_BAT_213"]
source_types = ["SBG_23"]

# type of detector / UHECR dataset
detector_type = "TA2015"

In [2]:
# change detector property imports based on detector type string
if detector_type == "TA2015":
    from fancy.detector.TA2015 import detector_properties, Eth
elif detector_type == "auger2014":
    from fancy.detector.auger2014 import detector_properties, Eth
elif detector_type == "auger2010":
    from fancy.detector.auger2010 import detector_properties, Eth
else:
    raise Exception("Undefined detector type!")

In [3]:
'''Precompute exposure integral table and energy grid'''

for source_type in source_types:
    print("Current Source: {0}".format(source_type))

    # File in which to store precomputation
    # create new files with TA label
    table_file = "tables_{0}_{1}.h5".format(source_type, detector_type)

    data = Data()
    data.add_source(source_file, source_type)  
    data.add_detector(detector_properties)  # KW: add detector information

    model_name = 'joint_model.stan'
    model = Model(model_filename = model_name, include_paths = stan_path)
    model.input(Eth = Eth) # EeV

    summary = b'Precomputation for tables'
    analysis = Analysis(data, model, analysis_type = 'joint', 
                        filename = table_file, summary = summary)

    print("Evaluating Exposure Integral...")
    analysis.build_tables(fit_only = True)
    analysis.tables.save(table_file)

    print("Evaluating Arrival Energies...")
    analysis.build_energy_table(table_file = table_file)

Current Source: SBG_23
Evaluating Exposure Integral...


Precomputing exposure integral: 100%|██████████| 23/23 [01:41<00:00,  4.40s/it]


Evaluating Arrival Energies...



Precomputing energy grids: 100%|██████████| 23/23 [00:55<00:00,  2.41s/it]


In [4]:
# check if the table is constructed correctly:

for source_type in source_types:
    print("Current Source: {0}".format(source_type))

    # File in which to store precomputation
    # create new files with TA label
    table_file = "tables_{0}_{1}.h5".format(source_type, detector_type)

    with h5py.File(table_file, "r") as f:
        print(f.keys())

        print("Exposure integral keys:")
        print(f["main"].keys())   # main exposure integral table

        print("Energy Grid keys:")
        print(f["energy"].keys())

Current Source: SBG_23
<KeysViewHDF5 ['energy', 'main', 'params', 'simulation', 'varpi']>
Exposure integral keys:
<KeysViewHDF5 ['kappa', 'table']>
Energy Grid keys:
<KeysViewHDF5 ['E_grid', 'Earr_grid']>
